In [1]:
from openpyxl.reader.excel import load_workbook
import pandas as pd 
import numpy as np
from itertools import islice
from openpyxl.utils import FORMULAE
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Border, Side
from openpyxl.formatting.rule import ColorScaleRule, CellIsRule, FormulaRule
import statistics
from openpyxl.chart import (ScatterChart, Reference, Series)

In [2]:
# wb = load_workbook('sample.xlsx')

filename = 'sample.xlsx'
wb = load_workbook(filename)
ws = wb['Sheet1']

#create df of plate map
samp = ws.values
columns = next(samp)[1:]
samp = list(samp)
idx = [r[0] for r in samp]
samp = (islice(r, 1, None) for r in samp)
df = pd.DataFrame(samp, index=idx, columns=columns)

#create df of raw data values
ws2 = wb['Sheet2']

data = ws2.values
columns = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)
df2 = pd.DataFrame(data, index=idx, columns=columns)

print(df, '\n', df2)

                  1       2        3       4       5       6       7       8   \
A                 S1      S2       S3      S4      S5      S6      S7      S8   
B                 S1      S2       S3      S4      S5      S6      S7      S8   
C                S13     S14      S15     S16     S17     S18     S19     S20   
D                S13     S14      S15     S16     S17     S18     S19     S20   
E                S25     S26      S27     S28     S29     S30     S31     S32   
F                S25     S26      S27     S28     S29     S30     S31     S32   
G                  A       B        C       D       E       F       G       H   
H                  A       B        C       D       E       F       G       H   
NaN             None    None     None    None    None    None    None    None   
NaN                1       2        3       4       5       6       7       8   
Raw Data           1       2        3       4       5       6       7       8   
A            0.70000  0.6000

In [3]:
#find position of specific value in df

def findPosInDf(dfIn,findme):
    positions = []
    irow =0
    while ( irow < len(dfIn.index)):
        list_colPositions=dfIn.columns[dfIn.iloc[irow,:]==findme].tolist()   
        if list_colPositions != []:
            colu_iloc = dfIn.columns.get_loc(list_colPositions[0])
            positions.append([irow  , colu_iloc ])
        irow +=1

    return positions


#print(findPosInDf(df, 'I'))

bgPos = findPosInDf(df, 'I')

#loop to give value at every position of 'I'
value = []
for x in bgPos:  
    value.append(df2.iat[x[0],x[1]]) #returns all values for I
   
#print(value)


In [4]:
#avg all bgPos values from loop

def avg(value):
    return sum(value) / len(value)

avg(value)
bgAvg = avg(value)

In [5]:
#paste df2 under df in first excel sheet

rows = dataframe_to_rows(df2)

for r_idx, row in enumerate(rows, 11):
    for c_idx, value in enumerate(row, 1):
        ws.cell(row=r_idx, column=c_idx, value=value)


In [6]:
#input table titles into excel
ws['A1'] = 'Plate Map'
ws['A12'] = 'Raw Data'

ws['A22'] = 'Avg BG:'
ws['B22'] = bgAvg

ws['A24'] = 'BG Subtract'

ws['A33'] = 'Mean Std:'
ws['A34'] = 'ug/ml:'
wb.save(filename)

In [7]:
#format border around cells

In [8]:
#subtract bgAvg from all df2 cells
s = 0
subData = []
for s in df2:
    subData = df2 - bgAvg
    s+=1
#print(subData)


In [9]:
#paste subtracted data into excel
rows = dataframe_to_rows(subData)

for r_idx, row in enumerate(rows, 23):
    for c_idx, value in enumerate(row, 1):
        ws.cell(row=r_idx, column=c_idx, value=value)

wb.save(filename)

In [10]:
#identify standards 

stA = findPosInDf(df, 'A')
stB = findPosInDf(df, 'B')
stC = findPosInDf(df, 'C')
stD = findPosInDf(df, 'D')
stE = findPosInDf(df, 'E')
stF = findPosInDf(df, 'F')
stG = findPosInDf(df, 'G')
stH = findPosInDf(df, 'H')

stAll = [stA, stB, stC, stD, stE, stF, stG, stH]  #list of all std positions


In [11]:
stSep = []

for x in stAll: #isolates 1st std replicates from all std list
    stSep.append(x[0])
    std1 = []
    for x in stSep: #gets value of replicate 1 from bg subtracted data 
       std1.append(subData.iat[x[0],x[1]])
       
print(std1)
   
        
stSep2 = []

for x in stAll: #isolates 2ns std replicates from all std list
    stSep2.append(x[1])
    std2 = []
    for x in stSep2: #gets value of replicate 2 from bg subtracted data 
       std2.append(subData.iat[x[0],x[1]])
       
print(std2)

[1.8551, 1.4045999999999998, 0.8961, 0.6940999999999999, 0.46260000000000007, 0.22310000000000002, 0.1286, 0.031100000000000003]
[1.8768, 1.3936, 0.9056, 0.6658999999999999, 0.5065999999999999, 0.22350000000000003, 0.1436, 0.04609999999999999]


In [12]:
#mean value for each std
a = std1
b = std2
avgStd = [statistics.mean(k) for k in zip(a, b)]
print(avgStd)

[1.86595, 1.3990999999999998, 0.9008499999999999, 0.6799999999999999, 0.48460000000000003, 0.22330000000000003, 0.1361, 0.038599999999999995]


In [13]:
#put list of std means into excel
row = 33

for i, value in enumerate(avgStd):
    ws.cell(column=i+2, row=row, value=value)
wb.save(filename)

In [14]:
#put known std values into excel
stKnownVal = [2000, 1500, 1000, 750, 500, 250, 125, 50]

row = 34

for i, value in enumerate(stKnownVal):
    ws.cell(column=i+2, row=row, value=value)
wb.save(filename)

In [15]:
#std curve scatterplot

#rows = ['ug/ml', 'Mean Std'], [ws['B34':'I34'], ws['B33':'I33']]

#for row in rows: 
 #   ws.append(row) 
 
chart = ScatterChart() 
  
# create data for plotting 
xvalues = Reference(ws, min_col = 2, 
                    max_col = 9, min_row = 34) 
                      
yvalues = Reference(ws, min_col = 2, 
                    max_col = 9, min_row = 33) 
                      
size = Reference(ws, min_col = 3, 
                 min_row = 2, max_row = 5) 
  
# create a 1st series of data 
series = Series(values = yvalues, xvalues = xvalues, 
                      zvalues = size, title ="Std Curve") 
  
# add series data to the chart object 
chart.series.append(series) 
  
# set the title of the chart 
chart.title = "Std Curve "
  
# set the title of the x-axis 
chart.x_axis.title = " Mean Abs "
  
# set the title of the y-axis 
chart.y_axis.title = " ug/ml Protein "
  
# add chart to the sheet 
# the top-left corner of a chart 
# is anchored to cell E2 . 
ws.add_chart(chart, "B40") 
  
# save the file 
wb.save(filename) 